In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

#### Este dataset engloba informações sobre reembolsos de despesas de senadores entre os anos de 2008 e 2022

In [67]:
# Carregar o dataset usando ponto e vírgula como delimitador
df = pd.read_csv('dataset_senadores.csv', sep=';')

In [68]:
df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2022,1,NELSINHO TRAD,"ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, C...",465.789.351-34,FÁBIO MARCELO DE PAULI,41,2022-01-01,REFERENTE AO PAGAMENTO DO ALUGUEL NO MÊS DE JA...,3500.00,2173482
1,2022,1,ROSE DE FREITAS,"ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, C...",40.432.544/0001-47,CLARO S.A,5080033843422,2022-01-01,DESCONHECIDO,147.10,2174453
2,2022,1,DÁRIO BERGER,"ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, C...",02.558.157/0001-62,VIVO - TELEFONICA S.A.,14175407140,2022-01-01,DESCONHECIDO,164.98,2173788
3,2022,1,HUMBERTO COSTA,"CONTRATAÇÃO DE CONSULTORIAS, ASSESSORIAS, PESQ...",43.382.036/0001-90,MAPEO SERVIÇOS DE INOVAÇÃO E TECNOLOGIA LTDA,00000093,2022-01-01,APOIO TÉCNICO EM TRATAMENTO DE DADOS E HOSPEDA...,1250.00,2174078
4,2022,1,MARCOS DO VAL,"ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, C...",31.710.825/0001-23,ATELIER LOCAÇÃO PATRIMONIAL LTDA,01/2022,2022-01-01,DESCONHECIDO,6500.00,2173024


In [69]:
# Exibir as informações gerais do dataset
df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317961 entries, 0 to 317960
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                317961 non-null  int64  
 1   MES                317961 non-null  int64  
 2   SENADOR            317961 non-null  object 
 3   TIPO_DESPESA       317961 non-null  object 
 4   CNPJ_CPF           317961 non-null  object 
 5   FORNECEDOR         317961 non-null  object 
 6   DOCUMENTO          317958 non-null  object 
 7   DATA               317961 non-null  object 
 8   DETALHAMENTO       317961 non-null  object 
 9   VALOR_REEMBOLSADO  317961 non-null  float64
 10  COD_DOCUMENTO      317961 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 26.7+ MB


(None,
                  ANO            MES  VALOR_REEMBOLSADO  COD_DOCUMENTO
 count  317961.000000  317961.000000      317961.000000   3.179610e+05
 mean     2015.153915       6.644268        1019.058547   1.985928e+10
 std         3.746790       3.308163        2599.397652   1.987107e+11
 min      2008.000000       1.000000       -1500.000000   1.438800e+05
 25%      2012.000000       4.000000         100.000000   6.211970e+05
 50%      2015.000000       7.000000         283.480000   2.019630e+06
 75%      2018.000000      10.000000        1000.000000   2.106125e+06
 max      2022.000000      12.000000      270000.000000   2.009031e+12)

In [70]:
# Verificar a quantidade de valores ausentes por coluna
df.isnull().sum()

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DOCUMENTO            3
DATA                 0
DETALHAMENTO         0
VALOR_REEMBOLSADO    0
COD_DOCUMENTO        0
dtype: int64

In [71]:
# 3 valores ausentes na coluna "DOCUMENTO"
df = df.dropna(subset=['DOCUMENTO'])

print(df.isnull().sum()) # Sem mais valores ausentes

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DOCUMENTO            0
DATA                 0
DETALHAMENTO         0
VALOR_REEMBOLSADO    0
COD_DOCUMENTO        0
dtype: int64


In [72]:
# Reconvertendo a coluna 'DATA' com tratamento de erros
df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')

# Verificar se houve problemas na conversão
invalid_dates = df['DATA'].isnull().sum()
print("Datas inválidas após conversão:", invalid_dates)

Datas inválidas após conversão: 0


In [73]:
# Verificar o tamanho dos valores na coluna 'CNPJ_CPF'
df['CNPJ_CPF'].str.len().value_counts() # Existem numeros com 14 e 18 caracteres, incomparáveis com CNPJ e CPF

CNPJ_CPF
18    307181
14      7628
12      3144
17         5
Name: count, dtype: int64

In [74]:
# Expressões regulares para CNPJ e CPF
cnpj_pattern = re.compile(r"^\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}$")
cpf_pattern = re.compile(r"^\d{3}\.\d{3}\.\d{3}-\d{2}$")

# Identificar valores que NÃO seguem nenhum dos padrões
invalid_cnpj_cpf = df[
    ~df['CNPJ_CPF'].str.match(cnpj_pattern) & 
    ~df['CNPJ_CPF'].str.match(cpf_pattern)
]

# Exibir alguns exemplos desses valores
print("Valores com formatos incorretos de CNPJ/CPF:\n", invalid_cnpj_cpf['CNPJ_CPF'].unique()) # Há CNPJs e CPFs com formatos incorretos

Valores com formatos incorretos de CNPJ/CPF:
 ['0.030.659/7003-38' '0.420.605/0008-08' '7.388.199/7000-10'
 '0.240.900/0000-25' '0.478.056/5000-43' 'DESCONHECIDO']


In [75]:
# Função para limpar e formatar CPF/CNPJ
def formatar_cnpj_cpf(valor):
    # Remover pontos, barras e hífens
    valor = re.sub(r'[^0-9]', '', str(valor))  # Remove qualquer coisa que não seja número
    
    if len(valor) == 11:  # CPF
        # Verificar se o CPF é válido (apenas para consistência de tamanho)
        return f"{valor[:3]}.{valor[3:6]}.{valor[6:9]}-{valor[9:]}"
    
    elif len(valor) == 14:  # CNPJ
        # Verificar se o CNPJ é válido (apenas para consistência de tamanho)
        return f"{valor[:2]}.{valor[2:5]}.{valor[5:8]}/{valor[8:12]}-{valor[12:]}"
    
    return np.nan  # Caso não seja nem CPF nem CNPJ, retorna NaN (inconsistente)

# Aplicar a função para limpar os dados
df['CNPJ_CPF'] = df['CNPJ_CPF'].apply(formatar_cnpj_cpf)

# Identificar valores que ainda são inválidos (nao são nem CNPJ nem CPF)
invalid_cnpj_cpf_after_clean = df[df['CNPJ_CPF'].isna()]

# Exibir valores ainda incorretos
print("Valores ainda incorretos após limpeza e formatação:\n", invalid_cnpj_cpf_after_clean['CNPJ_CPF'].unique())

Valores ainda incorretos após limpeza e formatação:
 [nan]


In [76]:
df['CNPJ_CPF'].fillna('DESCONHECIDO', inplace=True) # Preencher valores nulos com 'DESCONHECIDO'

C:\Users\higor\AppData\Local\Temp\ipykernel_5124\853280785.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CNPJ_CPF'].fillna('DESCONHECIDO', inplace=True) # Preencher valores nulos com 'DESCONHECIDO'


In [77]:
# Padronizar textos e remover espaços extras
text_columns = ['SENADOR', 'TIPO_DESPESA', 'FORNECEDOR', 'DETALHAMENTO']
df[text_columns] = df[text_columns].apply(lambda col: col.str.strip().str.upper())


In [78]:
df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 317958 entries, 0 to 317960
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                317958 non-null  int64         
 1   MES                317958 non-null  int64         
 2   SENADOR            317958 non-null  object        
 3   TIPO_DESPESA       317958 non-null  object        
 4   CNPJ_CPF           317958 non-null  object        
 5   FORNECEDOR         317958 non-null  object        
 6   DOCUMENTO          317958 non-null  object        
 7   DATA               317958 non-null  datetime64[ns]
 8   DETALHAMENTO       317958 non-null  object        
 9   VALOR_REEMBOLSADO  317958 non-null  float64       
 10  COD_DOCUMENTO      317958 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 29.1+ MB


(None,
                  ANO            MES                           DATA  \
 count  317958.000000  317958.000000                         317958   
 mean     2015.153945       6.644283  2015-08-29 22:51:39.533901056   
 min      2008.000000       1.000000            2008-02-01 00:00:00   
 25%      2012.000000       4.000000            2012-07-18 00:00:00   
 50%      2015.000000       7.000000            2015-08-03 00:00:00   
 75%      2018.000000      10.000000            2018-07-05 00:00:00   
 max      2022.000000      12.000000            2022-12-31 00:00:00   
 std         3.746794       3.308168                            NaN   
 
        VALOR_REEMBOLSADO  COD_DOCUMENTO  
 count      317958.000000   3.179580e+05  
 mean         1019.068033   1.985947e+10  
 min         -1500.000000   1.438800e+05  
 25%           100.000000   6.211982e+05  
 50%           283.500000   2.019632e+06  
 75%          1000.000000   2.106126e+06  
 max        270000.000000   2.009031e+12  
 std    

In [79]:
# Verificar duplicados
duplicados = df.duplicated()
print("Quantidade de registros duplicados:", duplicados.sum())

# Exibir alguns exemplos, se houver
if duplicados.sum() > 0:
    display(df[duplicados].head())

# Remover duplicados, mantendo o primeiro registro
df = df.drop_duplicates()


Quantidade de registros duplicados: 0


In [80]:
# Verificar a quantidade de valores negativos
valores_negativos = df[df['VALOR_REEMBOLSADO'] < 0]
print("Quantidade de valores negativos em VALOR_REEMBOLSADO:", len(valores_negativos))

# Exibir exemplos para entender o contexto
display(valores_negativos.head())


Quantidade de valores negativos em VALOR_REEMBOLSADO: 3


,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
104445,2017,5,DÁRIO BERGER,"CONTRATAÇÃO DE CONSULTORIAS, ASSESSORIAS, PESQ...",73.228.876/0001-63,TV CLIPAGEM LTDA. EPP.,001661,2017-06-02,DESPESA COM MONITORAMENTO DE INFORMAÇÃO JORNAL...,-243.4,2068568
148529,2015,1,RANDOLFE RODRIGUES,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",02.012.862/0001-60,TAM LINHAS AEREAS S/A,9572104373256,2015-01-28,"9572100929351 - R$ 1.298,93 (1º BILHETE EMITID...",-9.0,2000358
148873,2015,2,GLADSON CAMELI,"ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, C...",63.600.449/0001-00,ARRAS ADM. DE BENS IMÓVEIS LIMPEZA E CONSERVAÇ...,DESCONHECIDO,2015-01-15,CONTRATO DE LOCAÇÃO DE IMÓVEL COMERCIAL DO ESC...,-1500.0,2001228


In [81]:
# Existem apenas 3 valores negativos, que parecem ser erros de digitação, podem ser removidos
df = df[df['VALOR_REEMBOLSADO'] >= 0]

In [82]:
# Verificar unicidade
cod_unicos = df['COD_DOCUMENTO'].is_unique
print("Códigos de documento são únicos?", cod_unicos)

# Verificar se há valores ausentes
cod_nulos = df['COD_DOCUMENTO'].isnull().sum()
print("Quantidade de valores nulos em COD_DOCUMENTO:", cod_nulos) # Não há limpeza a ser feita


Códigos de documento são únicos? True
Quantidade de valores nulos em COD_DOCUMENTO: 0


In [83]:
# Verificar valores únicos
tipos_despesas = df['TIPO_DESPESA'].unique()
print("Tipos de Despesa:", tipos_despesas)

# Contagem de cada categoria
df['TIPO_DESPESA'].value_counts()


Tipos de Despesa: ['ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, COMPREENDENDO DESPESAS CONCERNENTES A ELES.'
 'CONTRATAÇÃO DE CONSULTORIAS, ASSESSORIAS, PESQUISAS, TRABALHOS TÉCNICOS E OUTROS SERVIÇOS DE APOIO AO EXERCÍCIO DO MANDATO PARLAMENTAR'
 'PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIONAIS'
 'AQUISIÇÃO DE MATERIAL DE CONSUMO PARA USO NO ESCRITÓRIO POLÍTICO, INCLUSIVE AQUISIÇÃO OU LOCAÇÃO DE SOFTWARE, DESPESAS POSTAIS, AQUISIÇÃO DE PUBLICAÇÕES, LOCAÇÃO DE MÓVEIS E DE EQUIPAMENTOS.'
 'LOCOMOÇÃO, HOSPEDAGEM, ALIMENTAÇÃO, COMBUSTÍVEIS E LUBRIFICANTES'
 'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR' 'SERVIÇOS DE SEGURANÇA PRIVADA']


TIPO_DESPESA
LOCOMOÇÃO, HOSPEDAGEM, ALIMENTAÇÃO, COMBUSTÍVEIS E LUBRIFICANTES                                                                                                                                  133063
PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIONAIS                                                                                                                                                 68760
ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, COMPREENDENDO DESPESAS CONCERNENTES A ELES.                                                                                                           62624
AQUISIÇÃO DE MATERIAL DE CONSUMO PARA USO NO ESCRITÓRIO POLÍTICO, INCLUSIVE AQUISIÇÃO OU LOCAÇÃO DE SOFTWARE, DESPESAS POSTAIS, AQUISIÇÃO DE PUBLICAÇÕES, LOCAÇÃO DE MÓVEIS E DE EQUIPAMENTOS.     25838
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR                                                                                                                                                    

In [84]:
# Verificar quantidade total de registros após tratamento
print("Quantidade de registros após tratamento:", len(df))

# Verificar somatório de VALOR_REEMBOLSADO
print("Somatório de VALOR_REEMBOLSADO:", df['VALOR_REEMBOLSADO'].sum())


Quantidade de registros após tratamento: 317955
Somatório de VALOR_REEMBOLSADO: 324022586.0200001


In [85]:
# Exibir registros com valores negativos
negativos = df[df['VALOR_REEMBOLSADO'] < 0]
print(negativos)

# Opção 1: Converter para valores absolutos (caso sejam estornos)
df['VALOR_REEMBOLSADO'] = df['VALOR_REEMBOLSADO'].abs()

# Opção 2: Substituir valores negativos por zero
df.loc[df['VALOR_REEMBOLSADO'] < 0, 'VALOR_REEMBOLSADO'] = 0


Empty DataFrame
Columns: [ANO, MES, SENADOR, TIPO_DESPESA, CNPJ_CPF, FORNECEDOR, DOCUMENTO, DATA, DETALHAMENTO, VALOR_REEMBOLSADO, COD_DOCUMENTO]
Index: []
